#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2018


# Homework 3:  Embeddings + Recommenders

### 100 points [5% of your final grade]

### Due: Monday, April 9 by 11:59pm

*Goals of this homework:* There are two main learning objectives: (i) implement and evaluate a pre-cursor to modern word2vec embeddings; and (ii) implement, evaluate, and improve upon traditional collaborative filtering recommenders.

*Submission Instructions:* To submit your homework, rename this notebook as UIN_hw#.ipynb. For example, this homework submission would be: YourUIN_hw3.ipynb. Submit this notebook via ecampus. Your notebook should be completely self-contained, with the results visible in the notebook. 

*Late submission policy:* For this homework, you may use up to three of your late days, meaning that no submissions will be accepted after Thursday, April 12 at 11:59pm.

# Part 1: Word Embeddings (50 points)
For this first part, we're going to implement a word embedding approach that is a bit simpler than word2vec. The key idea is to look at co-occurrences between center words and context words (somewhat like in word2vec) but without any pesky learning of model parameters.

If you're interested in a deeper treatment of comparing count vs. learned embeddings, take a look at: [Don’t count, predict! A systematic comparison of
context-counting vs. context-predicting semantic vectors](
http://www.aclweb.org/anthology/P14-1023)

## Load the Brown Corpus

The dataset for this part is the (in)famous [Brown corpus](https://en.wikipedia.org/wiki/Brown_Corpus) that is a collection of text samples from a wide range of sources, with over one million unique words. Good for us, you can find the Brown corpus in nltk. *Make sure you have already installed nltk with something like: conda install nltk*

In [16]:
import nltk
nltk.download('brown')

[nltk_data] Downloading package brown to /Users/nikhil/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

Once you have it locally, you can load the dataset into your notebook. You can access the words using brown.words():

In [17]:
from nltk.corpus import brown
brown.words()


[u'The', u'Fulton', u'County', u'Grand', u'Jury', ...]

## 1.1 Dataset Pre-processing
OK, now we need to do some basic pre-processing. For this part you should:

* Remove stopwords and punctuation.
* Make everything lowercase.

Then, count how often each word occurs. We will define the 5,000 most  frequent words as your vocabulary (V). We will define the 1,000 most frequent words as our context (C). Include a print statement below to show the top-20 words after pre-processing.

In [18]:
import nltk
from nltk.corpus import brown
from nltk.corpus import stopwords
import string 
import numpy as np

stop_words = set(stopwords.words('english'))

lower_raw_words = set(w.lower() for w in brown.words())
raw_sents =  brown.sents()


raw_vocab = []
raw_vocab = lower_raw_words - stop_words 
vocab = set(raw_vocab) - set(string.punctuation)- set(['\'\'','``','--'])


sents =[]
for s in raw_sents:
    lower_words=[]
    for w in s:
        if w.lower() in vocab:
            lower_words.append(w.lower()) 
    sents.append(lower_words)

#print sents[1]

##########    define Target and context words ###############

all_words =  [item for sublist in sents for item in sublist]

fdist = nltk.FreqDist(all_words)

raw_targets = fdist.most_common(5000)
raw_context = fdist.most_common(1000) 

targets =[]
for w in raw_targets:
    targets.append(w[0])

context = []
for w in raw_context:
    context.append(w[0])


print targets[0:20]
print context[0:20]



[u'one', u'would', u'said', u'new', u'could', u'time', u'two', u'may', u'first', u'like', u'man', u'even', u'made', u'also', u'many', u'must', u'af', u'back', u'years', u'much']
[u'one', u'would', u'said', u'new', u'could', u'time', u'two', u'may', u'first', u'like', u'man', u'even', u'made', u'also', u'many', u'must', u'af', u'back', u'years', u'much']


## 1.2 Building the Co-occurrence Matrix 

For each word in the vocabulary (w), we want to calculate how often context words from C appear in its surrounding window of size 4 (two words before and two words after).

In other words, we need to define a co-occurrence matrix that has a dimension of |V|x|C| such that each cell (w,c) represents the number of times c occurs in a window around w. 

In [19]:
###########   calculate co-ocurrence matrix #################

co_mat = np.zeros((5000,1000))

for s in sents:
    for i in range (0, len(s)): 
        targetw = s[i]
        contextw=[]

        if (i-2>=0):
            contextw.append(s[i-2])
        if (i-1>=0):
            contextw.append(s[i-1])
        if (i+1<len(s)):
            contextw.append(s[i+1])
        if (i+2<len(s)):
            contextw.append(s[i+2])

        #print targetw 
        #print contextw
        

        if targetw in targets:
            for cw in contextw:
                if cw in context:
                    #print targets.index(targetw)
                    #print context.index(cw)
                    co_mat[targets.index(targetw),context.index(cw)]+=1


## 1.3 Probability Distribution

Using the co-occurrence matrix, we can compute the probability distribution Pr(c|w) of context word c around w as well as the overall probability distribution of each context word c with Pr(c).  

## 1.4 Embedding Representation

Now you can represent each vocabulary word as a |C| dimensional vector using this equation:

Vector(w)= max(0, log (Pr(c|w)/Pr(c)))

This is a traditional approach called *pointwise mutual information* that pre-dates word2vec by some time. 

In [20]:
import math 

c_word= np.sum(co_mat,1)
c_cont = np.sum(co_mat,0)

prob_cont = c_cont / np.sum(c_cont)

vec=np.zeros((5000,1000))

#for i in range (0,5000):
#    for j in range (0,1000):
#        prob_cont_given_word = co_mat[i,j]/c_word[i]
#        vec[i,j] = prob_cont_given_word/prob_cont[j]
#
#
#print co_mat[2,[0,9]]
#print prob_cont[0:9]
#print c_word[0:4]
#print vec[2,[0,11]]

p_ij = np.zeros((5000,1000))
pmi_ij = np.zeros((5000,1000))
ppmi = np.zeros((5000,1000))


co_mat = co_mat +1   # Add 1 smoothing 

sum_freq = np.sum(co_mat)

temp1= np.sum(co_mat,1)
temp2= np.sum(co_mat,0) 

p_i = temp1.astype(float)/sum_freq       # word probability in all contexts 
p_j = temp2.astype(float)/sum_freq       # context probabilty 


for i in range (0,5000):
    for j in range (0,1000):
        p_ij[i,j] = co_mat[i,j]/sum_freq
        pmi_ij[i,j] = math.log( (p_ij[i,j] /(p_i[i]*p_j[j])),2)
        if (pmi_ij[i,j] >0):
            ppmi[i,j]=pmi_ij[i,j]
        else:
            ppmi[i,j] = 0



## 1.5 Analysis

So now we have some embeddings for each word. But are they meaningful? For this part, you should:

- First, cluster the vocabulary into 100 clusters using k-means. Look over the words in each cluster, can you see any relation beween words? Discuss your observations.

- Second, for the top-20 most frequent words, find the nearest neighbors using cosine distance (1- cosine similarity). Do the findings make sense? Discuss.

In [20]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=100).fit(ppmi)
bb=kmeans.labels_

In [53]:
def init_list_of_objects(size):
    list_of_objects = list()
    for i in range(0,size):
        list_of_objects.append( list() ) #different object reference each time
    return list_of_objects

clusters = init_list_of_objects(100)

for i in range (0,len(bb)):
    clusters[bb[i]].append(i)

[107, 117, 147, 189, 227, 228, 241, 290, 337, 424, 558, 561, 628, 636, 639, 661, 691, 743, 869, 889, 895, 925, 1220, 1310, 1490]


In [76]:
from __future__ import print_function
for i in range (0,10):
    temp= clusters[i]
    print('\n\ncluster: ')
    for j in range (0,len(clusters[i])):
        print(targets[temp[j]], end=', ')
        



cluster: 
end, next, early, began, week, half, period, full, century, late, fall, earlier, appeared, summer, evening, month, spring, fiscal, afternoon, season, spent, sunday, session, november, saturday, 

cluster: 
man, young, woman, 

cluster: 
music, feeling, cold, rest, fine, suddenly, girls, french, met, hot, beautiful, apparently, man's, blood, chief, audience, obviously, hit, moving, born, poor, sun, deep, married, bit, filled, suppose, giving, americans, names, pain, spoke, touch, bright, mary, seeing, died, named, strange, begin, fresh, explained, coffee, looks, nice, watching, killed, turning, send, somehow, wine, rain, realize, clean, warm, busy, alive, sounds, angry, 

cluster: 
two, three, five, 

cluster: 
outside, view, except, beyond, inside, reached, color, opened, corner, closed, entered, leaving, key, opposite, waited, starting, shut, 

cluster: 
never, ever, 

cluster: 
final, 

cluster: 
little, place, home, enough, 

cluster: 
city, president, york, university, 

In [84]:
from sklearn.metrics.pairwise import cosine_similarity
cos_sim=cosine_similarity(ppmi,ppmi)
for i in range (0,5000):
    cos_sim[i,i]=0
sort_sim = np.argsort(cos_sim,axis=1)
for i in range (0,20):
    print('\n'+targets[i], end='-\t')
    for j in range (0,5):
        print(targets[sort_sim[i,5000-j-1]], end=' ')
    


one-	another first time man every 
would-	could might much said things 
said-	say know think like would 
new-	city york central first yankees 
could-	would wanted might want said 
time-	one long still way would 
two-	three four several five many 
may-	might would must also even 
first-	one last time later second 
like-	said know good could thought 
man-	woman one little boy things 
even-	much still would felt seem 
made-	make one still even making 
also-	may even well one still 
many-	two several even among people 
must-	would may might could things 
af-	q **zg p function polynomial 
back-	away around along home went 
years-	days months weeks year minutes 
much-	would even things little better 

0.551765470117


# Part 2. Collaborative Filtering (50 points)

In this second part, you will implement collaborative filtering on the Netflix prize dataset -- don’t freak out, the provided sample dataset has only ~2000 items and ~28,000 users.

As background, read the paper [Empirical Analysis of Predictive Algorithms for Collaborative Filtering](https://arxiv.org/pdf/1301.7363.pdf) up to Section 2.1. Of course you can read further if you are interested, and you can also refer to the course slides for collaborative filtering.

## 2.1 Load Netflix Data

The dataset is subset of movie ratings data from the Netflix Prize Challenge. Download the dataset from Piazza. It contains a train set, test set, movie file, and README file. The last two files are original ones from the Netflix Prize, however; in this homework you will deal with train and test files which both are subsets of the Netflix training data. Each of train and test files has lines having this format: MovieID,UserID,Rating.

Your job is to predict a rating in the test set using those provided in the training set.

In [1]:
# load the data, then print out the number of ratings, 
# movies and users in each of train and test sets.
# Your Code Here...
import numpy as np
np.seterr(divide='ignore', invalid='ignore')   # Ignore warning generated when 'NAN' values are created. NAN values are created intentionally for unavailable ratings


fp = open('./netflix-dataset/TrainingRatings.txt')
lines=fp.readlines()
fp.close()
temp = ([line.split(',') for line in lines])
train_movie =  [tempo[0] for tempo in temp]
train_user =   [tempo[1] for tempo in temp]
train_rating = [tempo[2] for tempo in temp]
print 'Training Database:'
print 'No of ratings = '+str(len(train_rating))
print 'No of unique movies = '+str(len(set(train_movie)))
print 'No of unique users = '+str(len(set(train_user)))


fp = open('./netflix-dataset/TestingRatings.txt')
lines=fp.readlines()
fp.close()
temp = ([line.split(',') for line in lines])
test_movie =  [tempo[0] for tempo in temp]
test_user =   [tempo[1] for tempo in temp]
test_rating = [tempo[2] for tempo in temp]
print '\nTesting Database:'
print 'No of ratings = '+str(len(test_rating))
print 'No of unique movies = '+str(len(set(test_movie)))
print 'No of unique users = '+str(len(set(test_user)))



Training Database:
No of ratings = 3255352
No of unique movies = 1821
No of unique users = 28978

Testing Database:
No of ratings = 100478
No of unique movies = 1701
No of unique users = 27555


## 2.2 Implement CF

In this part, you will implement the basic collaborative filtering algorithm described in Section 2.1 of the paper -- that is, focus only on Equations 1 and 2 (where Equation 2 is just the Pearson correlation). You should consider the first 5,000 users with their associated items in the test set. 

Note that you should test the algorithm for a small set of users e.g., 10 users first and then run for 5,000 users. It may take long to run but you won't have memory issues. 

Set k to 0.1. 

In [2]:
# Your Code Here....

# Mapping between Movie Ids to smaller unique integers 

def map_ids_to_smaller_nos (listo):
    dicto = {}
    no=0
    for ido in listo:
        if str(ido) not in dicto.keys():
            dicto[str(ido)]=no
            no=no+1
    return(dicto)



train_movie_dicto = map_ids_to_smaller_nos(train_movie)
train_user_dicto =  map_ids_to_smaller_nos(train_user)

# Self checking loop to ensure Map function is performing correctly, ( Maximum of mapped id should be equal to No of unique Ids -1 )

if (max(train_movie_dicto.values()) != len(set(train_movie))-1):
    print 'Error in Mapping \n'
if (max(train_user_dicto.values()) != len(set(train_user))-1):
    print 'Error in Mapping \n'

In [3]:
import numpy as np
no_movies=len(set(train_movie))
no_users=len(set(train_user))


V=np.zeros((no_users,no_movies))

for i in range (0,len(train_movie)):
    V[int(train_user_dicto[train_user[i]]), int(train_movie_dicto[train_movie[i]])]=train_rating[i]
    

In [197]:
# Following is NAIVE IMPLEMENTATION : 
#Initially I coded using following approach, then I realized that code takes forever to execute.
# Nevertheless, I am keeping the code intact here

from sklearn.metrics.pairwise import cosine_similarity
import math 
import time
def zero_to_nan (vec_i):
    for jjj in range (0, len(vec_i)):
        if (vec_i[jjj] == 0):
            vec_i[jjj] = float('nan')
    return vec_i

def nan_to_zero (vec_i):
    for jjj in range (0, len(vec_i)):
        if (math.isnan(vec_i[jjj])):
            vec_i[jjj] = 0
    return vec_i


predicted_ratings = []


for ii in range(0,10):
    
    a = train_user_dicto[test_user[ii]] 
    j = train_movie_dicto[test_movie[ii]]
    vec_a = V[a,:]
    va_bar=np.mean(vec_a[vec_a!=0])
    vec_a1 = zero_to_nan(vec_a)
    vec_a2 = vec_a - va_bar
    vec_a3 = np.nan_to_num(vec_a)
    p_aj = 0
    sum_w = 0
    for i in range (0, V.shape[0]):
        if ((i != a) & (V[i,j] !=0)):
            vec_i = V[i,:]
            #print vec_i
            
            start_time = time.time()
            vi_bar = np.mean(vec_i[vec_i!=0])
            print("--- %s mean ---" % (time.time() - start_time))
            start_time = time.time()
            vec_i1=zero_to_nan(vec_i)
            print("--- %s zero2nan ---" % (time.time() - start_time))
            
            vec_i2 = vec_i - vi_bar
            vec_i3=np.nan_to_num(vec_i)
            w_ai=cosine_similarity(np.reshape(vec_a3,[1,-1]),np.reshape(vec_i3,[1,-1]))
            #print w_ai,k * w_ai * (V[i,j]-vi_bar)
            p_aj +=  w_ai * (V[i,j]-vi_bar)
            sum_w += w_ai
    p_aj = p_aj / sum_w      # Normalization - K factor (K CAN NOT be 0.1 - it should be used to maintain sum(w_ai) =1)
    p_aj = p_aj + va_bar
    print p_aj
    predicted_ratings.append(p_aj)
print predicted_ratings

[[ 3.06382195]]
[[ 3.02218815]]
[[ 2.55542432]]
[[ 2.82846739]]
[[ 2.67426335]]


KeyboardInterrupt: 

In [4]:
####  SMART  IMPLEMENTATION ###############

# I have converted everything into matrix form: 
# Basically the most important term in both equations is V(i,j) - Vi_mean  
# To compute this, I converted original V[i,j] matrix into mean substracted matrix.
# Trick here is to convert unknown ratings into "nan" so that those are not involved in mean computation (np.nanmean) and
#then convert them to zero so that their contribution in predicting the ratings becomes zero.

from sklearn.metrics.pairwise import cosine_similarity
M = V
M[M == 0] = np.nan
M_bar=np.nanmean(M,1)
M=np.add(M, np.reshape(-M_bar,[M.shape[0],-1]))
M=np.nan_to_num(M)
w = cosine_similarity(M,M)
print M[0,:]
print w.shape

[-1.45098039 -1.45098039  0.         ...,  0.          0.          0.        ]
(28978, 28978)


In [18]:
print M.shape

(28978, 1821)


In [11]:
w_test=[]
vnorm_test =[]
p_bar_test =[]
matching =[]
for ii in range(0,len(test_user)):    
#for ii in range(0,10):    
    a = train_user_dicto[test_user[ii]] 
    j = train_movie_dicto[test_movie[ii]]
    w_test.append(w[a,:])
    p_bar_test.append(M_bar[a])
    vnorm_test.append(M[:,j])
    matching.append(a)

w_test=np.array(w_test)
vnorm_test=np.array(vnorm_test)
#vnorm_test=np.transpose(vnorm_test)
p_bar_test=np.array(p_bar_test)

var =[] 
for i in range (0,w_test.shape[0]):
    var.append(np.sum(np.multiply(w_test[i,:],vnorm_test[i,:])))
var=np.array(var)

w_sum=np.sum(w_test,1)



In [12]:
var_norm=np.divide(var,w_sum)       # k is inverse of sum of all weights waj over all j 
var_norm[np.isnan(var_norm)] = 0
predict_ratings = np.add(p_bar_test,var_norm)


## 2.3 Evaluation 

You should evaluate your predictions using Mean Absolute Error and Root Mean Squared Error. 

In [13]:
# Mean Absolute Error

from sklearn.metrics import mean_absolute_error
true_ratings=[]
for r in test_rating: 
    true_ratings.append(float(r.translate(None, '\t\n ')))
true_ratings=np.array(true_ratings)

print 'Mean Absolute Error : '+str(mean_absolute_error(true_ratings, predict_ratings))

Mean Absolute Error : 0.772049309323


In [14]:
# Root Mean Squared Error
from sklearn.metrics import mean_squared_error
from math import sqrt
rms = sqrt(mean_squared_error(true_ratings, predict_ratings))
print 'RMS error = '+str(rms)

RMS error = 2.87543251544


In [7]:
# Getting the size of variables and deleting unwanted variables to avoid kernel crash

import sys
del train_user, train_movie, train_rating, temp, lines, w
for var, obj in locals().items():
    print var, sys.getsizeof(obj)


_dh 80
__ 37
test_user 824472
train_user_dicto 3146008
__builtin__ 56
no_users 24
cosine_similarity 120
line 55
quit 64
_i7 322
_i6 206
_i5 186
_i4 1436
_i3 566
_i2 1472
_i1 2464
__package__ 16
exit 64
get_ipython 80
_i 206
np 56
__doc__ 101
fp 144
test_movie 824472
map_ids_to_smaller_nos 120
__builtins__ 56
M 422151616
M_bar 231920
_ih 136
tempo 168
sys 56
var 40
V 422151616
__name__ 45
___ 37
_ 37
_sh 56
obj 280
test_rating 824472
i 24
no_movies 24
_iii 1436
_ii 186
In 136
train_movie_dicto 196888
_oh 280
Out 280


In [8]:
# saving additional variables to file . In case Kernel crashes. 

import pickle 
with open('objs1.pkl', 'w') as f:  # Python 3: open(..., 'wb')
    pickle.dump([ M, M_bar], f)

## 2.4 Extensions

Given your results in the previous part, can you do better? For this last part you should report on your best attempt at improving MAE and RMSE. Provide code, results, plus a brief discussion on your approach.

In [9]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds

#get SVD components from train matrix. Choose k.
u, s, vt = svds(M, k = 200)
s_diag_matrix=np.diag(s)
all_mat = np.dot(np.dot(u, s_diag_matrix), vt)

In [12]:
M_bar=np.reshape(M_bar,(-1,1))

In [13]:
all_mat = all_mat + M_bar
predict_ratings=[]
for ii in range(0,len(test_user)):    
    a = train_user_dicto[test_user[ii]] 
    j = train_movie_dicto[test_movie[ii]]
    predict_ratings.append(all_mat[a,j])

In [14]:
# Mean Absolute Error

from sklearn.metrics import mean_absolute_error
true_ratings=[]
for r in test_rating: 
    true_ratings.append(float(r.translate(None, '\t\n ')))
true_ratings=np.array(true_ratings)

print 'Mean Absolute Error : '+str(mean_absolute_error(true_ratings, predict_ratings))

# Root Mean Squared Error
from sklearn.metrics import mean_squared_error
from math import sqrt
rms = sqrt(mean_squared_error(true_ratings, predict_ratings))
print 'RMS error = '+str(rms)

Mean Absolute Error : 0.771227744546
RMS error = 0.968799625802


*Insert discussion here*